In [64]:
import pandas as pd
import numpy as np
import math
import geopandas as gpd
from shapely.geometry import Point

import sys, os
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("Clean_Process_data"), '..', 'functions/data_transforming')))
from spatial_analysis import *

import warnings
warnings.filterwarnings('ignore')

# Clip zip code shapefile to only SF

In [3]:
communities = gpd.read_file('../data/raw_data/zip_codes/geo_export_37348335-ac06-4ade-bf6d-453d393d539d.shp')

In [4]:
communities = communities[communities["po_name"] == "SAN FRANCISCO"]
communities = communities[communities["zip"] != '94130']
communities = communities[communities["zip"] != '94129']
communities = communities[communities["zip"] != '94128']

In [5]:
communities.to_file('../data/clean_data/SF_zip_codes/SF_zip_codes.shp')

# Clean/Process parks data

In [38]:
parks = pd.read_csv('../data/raw_data/parks_data.csv')

In [39]:
parks = parks.iloc[1:,:]
parks = parks.dropna(subset = ["Location 1"])

In [40]:
temp = parks["Location 1"].apply(lambda x: x.split('\n'))
parks["address"] = temp.apply(lambda x: x[0])
parks["city_state"] = temp.apply(lambda x: x[1])
parks["lat_long"] = temp.apply(lambda x: x[2])
del parks["Location 1"]
del parks["Lat"]
del parks["Zipcode"]

In [41]:
# Add zip code field to PARKS data using spatial join
parks = spatial_join_zipcode(parks, communities)

In [42]:
parks.to_csv('../data/clean_data/parks.csv', index=False)

# Clean/Process schools data

In [43]:
schools = pd.read_csv('../data/raw_data/Schools.csv')

In [44]:
schools = schools.dropna(subset = ["Location 1"])
temp = schools["Location 1"].apply(lambda x: x.split('\n'))
schools["State"] = temp.apply(lambda x: x[0])
schools["lat_long"] = temp.apply(lambda x: x[1])
del schools["Location 1"]
del schools["State"]

In [45]:
# Add zip code field to SCHOOLS data using spatial join
schools = spatial_join_zipcode(schools, communities)

In [46]:
schools.to_csv('../data/clean_data/schools.csv', index=False)

# Clean/Process bart stations data

In [47]:
bart_stations = gpd.read_file('../data/raw_data/bart_stations/BART_Sta_13.shp')

In [48]:
# Add zip code field to BART STATIONS data using spatial join
bart_stations = bart_stations.to_crs({'init': 'epsg:4326'})
bart_stations = gpd.sjoin(bart_stations, communities, how="inner", op='intersects')
del bart_stations["index_right"]

In [49]:
bart_stations.to_csv('../data/clean_data/bart_stations.csv', index=False)

# Process police department locations data

In [109]:
SFPD_locations = pd.read_csv('../data/raw_data/Police_Stations__2011_.csv')

In [113]:
SFPD_locations = SFPD_locations.rename(columns={'Location':'lat_long'})

In [114]:
# Add zip code field to SFPD LOCATIONS data using spatial join
SFPD_locations = spatial_join_zipcode(SFPD_locations, communities)

In [115]:
SFPD_locations.to_csv('../data/clean_data/Police_Stations.csv', index=False)

# Process police incident data

In [62]:
police_incidents = pd.read_csv('../data/raw_data/SFPD_Incidents_4052016.csv')

In [65]:
police_incidents = police_incidents.rename(columns={'Location':'lat_long'})

In [66]:
# add zip code field to POLICE INCIDENT data using spatial join
police_incidents = spatial_join_zipcode(police_incidents, communities)

In [69]:
police_incidents.to_csv('../data/clean_data/SFPD_Incidents_4052016.csv', index=False)

# Process zillow/citydata

In [70]:
zillow_citydata = pd.read_csv('../data/raw_data/zillow_citydata.csv')

In [100]:
zillow_citydata = zillow_citydata.rename(columns={'lontitude':'longitude'})
zillow_citydata["lat_long"] = zillow_citydata[["latitude", "longitude"]].apply(lambda x: str(tuple(x)), axis=1)

In [101]:
# add zip code field to ZILLOW/CITYDATA using spatial join
zillow_citydata = spatial_join_zipcode(zillow_citydata, communities)
del zillow_citydata["ZIP"]
del zillow_citydata["State"]
del zillow_citydata["City"]

In [107]:
zillow_citydata.to_csv('../data/clean_data/zillow_citydata.csv', index=False)

# Process restaurants data

In [162]:
restaurants = pd.read_csv('../data/raw_data/Restaurant.csv')

In [157]:
# restaurants = restaurants.rename(columns={'business_location':'lat_long'})
# restaurants = restaurants.dropna(subset = ["lat_long"])

# def remove_missing_lat_long(x):
#     if len(x) >= 3:
#         if x[2] != "":
#             return x[2]
#         else:
#             return np.nan
#     return np.nan

# temp = restaurants["Business Location"].apply(lambda x: x.split('\n'))
# restaurants["Address"] = temp.apply(lambda x: x[0])
# restaurants["City_State_Zipcode"] = temp.apply(lambda x: x[1])
# restaurants["lat_long"] = temp.apply(lambda x: remove_missing_lat_long(x))
# del restaurants["Business Location"]
# restaurants = restaurants.dropna(subset = ["lat_long"])

In [158]:
# add zip code field to RESTAURANTS data using spatial join
# restaurants = spatial_join_zipcode(restaurants, communities)

In [163]:
# merge communities data on zip code because a lot of latitude/longitude data is missing
restaurants = pd.merge(restaurants, communities, left_on="business_postal_code", right_on="zip", how="left")

In [164]:
restaurants.to_csv('../data/clean_data/restaurants.csv', index=False)

# Clean gold standard data

In [315]:
gold_standard = pd.read_csv('../data/raw_data/gold_standard.csv')

In [316]:
gold_standard["lat_long"] = gold_standard[["lat", "lon"]].apply(lambda x: str(tuple(x)), axis=1)

In [317]:
# add zip code field to GOLD STANDARD using spatial join
gold_standard = spatial_join_zipcode(gold_standard, communities)
gold_standard["zip"] = gold_standard["zip"].astype(np.int64)
gold_standard = gold_standard.drop(["location", "lat", "lon", "lat_long", "geometry", "area", "length", "po_name", "state", "greenery", "safety", "restaurants"], axis=1)

In [318]:
gold_standard = gold_standard.groupby(["zip"], as_index=False).mean()

In [319]:
# gold_standard["safety"][gold_standard["safety"] <= .8] = 0
# gold_standard["safety"][gold_standard["safety"] > .8] = 1
gold_standard["sanitation"][gold_standard["sanitation"] <= .6] = 0
gold_standard["sanitation"][gold_standard["sanitation"] > .6] = 1
gold_standard["peace_quiet"][gold_standard["peace_quiet"] <= .5] = 0
gold_standard["peace_quiet"][gold_standard["peace_quiet"] > .5] = 1
gold_standard["appearance"][gold_standard["appearance"] <= .5] = 0
gold_standard["appearance"][gold_standard["appearance"] > .5] = 1
gold_standard["children_friendly"][gold_standard["children_friendly"] <= .5] = 0
gold_standard["children_friendly"][gold_standard["children_friendly"] > .5] = 1
# gold_standard["greenery"][gold_standard["greenery"] < .5] = 0
# gold_standard["greenery"][gold_standard["greenery"] >= .5] = 1
gold_standard["walking_condition"][gold_standard["walking_condition"] <= .5] = 0
gold_standard["walking_condition"][gold_standard["walking_condition"] > .5] = 1
# gold_standard["restaurants"][gold_standard["restaurants"] <= .4] = 0
# gold_standard["restaurants"][gold_standard["restaurants"] > .4] = 1
gold_standard["coffee"][gold_standard["coffee"] < .4] = 0
gold_standard["coffee"][gold_standard["coffee"] >= .4] = 1
gold_standard["nightlife"][gold_standard["nightlife"] <= 0] = 0
gold_standard["nightlife"][gold_standard["nightlife"] > 0] = 1
gold_standard["dog_friendly"][gold_standard["dog_friendly"] < 1] = 0
gold_standard["dog_friendly"][gold_standard["dog_friendly"] >= 1] = 1
gold_standard["construction"][gold_standard["construction"] <= .1] = 0
gold_standard["construction"][gold_standard["construction"] > .1] = 1

In [320]:
parks = pd.read_csv('../data/clean_data/parks.csv')
schools = pd.read_csv('../data/clean_data/schools.csv')
bart_stations = pd.read_csv('../data/clean_data/bart_stations.csv')
# SFPD_locations = pd.read_csv('../data/clean_data/Police_Stations.csv')
SFPD_incidents = pd.read_csv('../data/clean_data/SFPD_Incidents_4052016.csv')
zillow_citydata = pd.read_csv('../data/clean_data/zillow_citydata.csv')
restaurants = pd.read_csv('../data/clean_data/restaurants.csv')

In [321]:
# adding parks - binary
temp = parks.groupby(["zip", "area"], as_index=False).size().reset_index().rename(columns={0:'parks'})
temp["parks"] = temp["parks"]/temp["area"]*1e7 # normalizing using the area of the zip code
del temp["area"]
temp["parks"][temp["parks"] <= 3] = 0
temp["parks"][temp["parks"] > 3] = 1
gold_standard = gold_standard.merge(temp, on="zip", how="left")

# adding schools - binary
temp = schools.groupby(["zip", "area"], as_index=False).size().reset_index().rename(columns={0:'schools'})
temp["schools"] = temp["schools"]/temp["area"]*1e7 # normalizing using the area of the zip code
del temp["area"]
temp["schools"][temp["schools"] <= 5] = 0
temp["schools"][temp["schools"] > 5] = 1
gold_standard = gold_standard.merge(temp, on="zip", how="left")

# adding bart_stations - binary
temp = bart_stations.groupby(["zip"], as_index=False).size().reset_index().rename(columns={0:'bart_stations'})
temp["bart_stations"][temp["bart_stations"] < 1] = 0
temp["bart_stations"][temp["bart_stations"] >= 1] = 1
gold_standard = gold_standard.merge(temp, on="zip", how="left")

# # adding SFPD_locations - binary
# temp = SFPD_locations.groupby(["zip"], as_index=False).size().reset_index().rename(columns={0:'SFPD_stations'})
# temp["SFPD_stations"][temp["SFPD_stations"] < 1] = 0
# temp["SFPD_stations"][temp["SFPD_stations"] >= 1] = 1
# gold_standard = gold_standard.merge(temp, on="zip", how="left")

# adding SFPD_incidents - binary
temp = SFPD_incidents.groupby(["zip", "area"], as_index=False).size().reset_index().rename(columns={0:'safety'})
temp["safety"] = temp["safety"]/temp["area"]*1e7 # normalizing using the area of the zip code
del temp["area"]
temp["safety"][temp["safety"] <= 1000] = 0
temp["safety"][temp["safety"] > 1000] = 1
gold_standard = gold_standard.merge(temp, on="zip", how="left")

# adding restaurants - binary
temp = restaurants.groupby(["business_postal_code", "business_name", "area"], as_index=False).size().reset_index().rename(columns={"business_postal_code":"zip", 0:'restaurants'})
temp = temp.groupby(["zip", "area"], as_index=False).size().reset_index().rename(columns={0:'restaurants'})
temp["restaurants"] = temp["restaurants"]/temp["area"]*1e7 # normalizing using the area of the zip code
del temp["area"]
temp["restaurants"][temp["restaurants"] <= 100] = 0
temp["restaurants"][temp["restaurants"] > 100] = 1
temp["zip"] = temp["zip"].astype(np.int64)
gold_standard = gold_standard.merge(temp, on="zip", how="left")

# adding zillow_citydata - binary
zillow_citydata["avg_rent"] = zillow_citydata["ZRent"]/zillow_citydata["sqft"]*450
temp = zillow_citydata.groupby(["zip"], as_index=False).mean()[["zip", "avg_rent"]]
gold_standard = gold_standard.merge(temp, on="zip", how="left")

# set all NaN values to 0
gold_standard = gold_standard.fillna(0)

In [322]:
gold_standard["similarity"] = 0
gold_standard = gold_standard[["zip","safety","sanitation","peace_quiet","appearance","children_friendly",
                              "walking_condition","restaurants","coffee","nightlife","dog_friendly",
                               "construction","parks","schools","bart_stations","avg_rent","similarity"]]
gold_standard.to_csv('../data/clean_data/gold_standard.csv', index=False)

In [323]:
gold_standard

,zip,safety,sanitation,peace_quiet,appearance,children_friendly,walking_condition,restaurants,coffee,nightlife,dog_friendly,construction,parks,schools,bart_stations,avg_rent,similarity
0,94102,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,2330.832106,0
1,94103,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,2615.436078,0
2,94104,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2420.167426,0
3,94105,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1949.499818,0
4,94107,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,2045.735398,0
5,94108,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,2652.314835,0
6,94109,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,2126.193504,0
7,94110,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,2442.880974,0
8,94111,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,2196.262982,0
9,94112,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,2713.020358,0
